In [39]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def get_job_links(url):
    job_links = []

    # Setup the Firefox WebDriver
    driver = webdriver.Firefox()

    driver.get(url)

    while True:
        try:
            # Handling cookie consent
            try:
                cookie_reject_button = WebDriverWait(driver, 5).until(
                    EC.element_to_be_clickable((By.XPATH, "//*[@id='onetrust-reject-all-handler']"))
                )
                cookie_reject_button.click()
            except Exception as e:
                pass

            # Wait for job links to load
            WebDriverWait(driver, 10).until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, "a[href*='/careers/jobs/job?jobid=']"))
            )

            # Find all job links on the current page
            jobs = driver.find_elements(By.CSS_SELECTOR, "a[href*='/careers/jobs/job?jobid=']")
            job_links.extend([job.get_attribute('href') for job in jobs])

            print(f"Scraping page: {driver.current_url}")  # Log the current URL

            # Find the 'Next' button by its text
            next_button = driver.find_element(By.XPATH, "//div[contains(text(), 'Next') and not(contains(@class, 'disabled'))]")

            if next_button:
                # Scroll to the 'Next' button and click using JavaScript
                driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
                driver.execute_script("window.scrollBy(0, -150);")  # Adjust if needed
                driver.execute_script("arguments[0].click();", next_button)

                # Wait for the page to load after clicking 'Next'
                WebDriverWait(driver, 10).until(
                    EC.presence_of_all_elements_located((By.CSS_SELECTOR, "a[href*='/careers/jobs/job?jobid=']"))
                )
            else:
                print(f"No 'Next' button found. Ending pagination at: {driver.current_url}")
                break

        except Exception as e:
            print("Error occurred on page:", driver.current_url)
            print("Error details:", e)
            break

    driver.quit()
    print(f"Total number of job links collected: {len(job_links)}")
    return job_links

# Starting URL
starting_url = 'https://www.mondelezinternational.com/careers/jobs/'
all_job_links = get_job_links(starting_url)
print(all_job_links)


Scraping page: https://www.mondelezinternational.com/careers/jobs/
Scraping page: https://www.mondelezinternational.com/careers/jobs/?term=&page=2
Scraping page: https://www.mondelezinternational.com/careers/jobs/?term=&page=3
Scraping page: https://www.mondelezinternational.com/careers/jobs/?term=&page=4
Scraping page: https://www.mondelezinternational.com/careers/jobs/?term=&page=5
Scraping page: https://www.mondelezinternational.com/careers/jobs/?term=&page=6
Scraping page: https://www.mondelezinternational.com/careers/jobs/?term=&page=7
Scraping page: https://www.mondelezinternational.com/careers/jobs/?term=&page=8
Scraping page: https://www.mondelezinternational.com/careers/jobs/?term=&page=9
Scraping page: https://www.mondelezinternational.com/careers/jobs/?term=&page=10
Scraping page: https://www.mondelezinternational.com/careers/jobs/?term=&page=11
Scraping page: https://www.mondelezinternational.com/careers/jobs/?term=&page=12
Scraping page: https://www.mondelezinternational.c